In [1]:
import json
import pandas as pd
import numpy as np

In [84]:
def dataframe_position(position):
    df1 = dataframe_position_version_deltagames(position, version_list[0])
    df2 = dataframe_position_version_deltagames(position, version_list[1])
    frames = [df1, df2]
    df = pd.concat(frames, ignore_index=True)
    df = df.sort_values(by='Games', ascending=False)
    
    Nsum = np.zeros(len(df.index))
    Pweight = np.zeros(len(df.index))
    Pmin = np.zeros(len(df.index))
    for i in range(len(df.index)):
        namei = df.iat[i, 0]
        Ri = df.iat[i, 1]
        Ni = df.iat[i, 2]
        Pi = df.iat[i, 3]
        Nsum[i] = Ni
        Pweight[i] = Pi * Ni
        Pmin[i] = Pi
        for j in range(i + 1,len(df.index)):
            namej = df.iat[j, 0]
            Rj = df.iat[j, 1]
            Nj = df.iat[j, 2]
            Pj = df.iat[j, 3]
            if namej == namei:
                print("%s %s %s %s - %s %s %s %s " % (namei, Ri, Ni, Pi, namej, Rj, Nj, Pj))
                Nsum[i] += Nj
                Pweight[i] += Nj * Pj
                if Pj < Pmin[i]:
                    Pmin[i] = Pj
                df.iat[j, 2] = 0
                df.iat[j, 3] = 0
        if Nsum[i] > 0:
            Pweight[i] = Pweight[i] / Nsum[i]
        
#         for idx_b, row_b in df[df.Games_sum < row_a.Games_sum].iterrows():
#             if row_a["Name"] == row_b["Name"]:
#                 df.at[idx_a, 'Games_sum'] = df.at[idx_a, 'Games_sum'] + row_b["Games_sum"]
                
#                 df.at[idx_a, 'Price_weight'] = df.at[idx_a, 'Price_weight'] + row_b["Price_weight"] * row_b["Games_sum"]
                
#                 if Pmin[i] > row_b["Price_weight"]:
#                     Pmin[i] = row_b["Price_weight"]
                
#                 df.at[idx_b, 'Games_sum'] = 0
#                 df.at[idx_b, 'Price_weight'] = 0
#                 print("%s %s %s %s  - %s %s %s %s " % (row_a["Name"], row_a["Rating"], row_a["Price_weight"], row_a["Games_sum"],
#                                                        row_b["Name"], row_b["Rating"], row_b["Price_weight"], row_b["Games_sum"]))
#         if df.at[idx_a, 'Games_sum'] > 0:
#             df.at[idx_a, 'Price_weight'] = df.at[idx_a, 'Price_weight'] / df.at[idx_a, 'Games_sum']

    
    df.rename(columns={'Price':'Price_weight'}, inplace=True)
    df.rename(columns={'Games':'Games_sum'}, inplace=True)
    
    df.Games_sum = Nsum
    df.Price_weight = Pweight
    df["Price_min"] = Pmin
    df.Price_min = df.Price_min.astype("int64")
    df["Value"] = value(df.Games_sum, df.Price_weight)
    df = df.sort_values(by='Value', ascending=False)
    return df

def value(games, price):
    P = np.array(price.to_list())
    N = np.array(games.to_list())
    V = (A + (1 - A) / (1 + np.exp(-a * (P - b)))) * (B * (c * N + d))
    Vmax = np.max(V)
    return V/Vmax

def import_scraped_data(time, position, version):
    filename = "../scraped_data/%s_%s_%s.json" % (time, position, version)
    with open(filename, encoding='utf-8') as json_data:
        d = json.load(json_data)
        json_data.close()
    return pd.DataFrame(d)

def dataframe_time_position_version(time, position, version):
    df = import_scraped_data(time, position, version)
    
    df = df.loc[:, ["Name", "Rating", "Games", "PS4"]]
    df.rename(columns={'PS4':'Price'}, inplace=True)
    df.Name = df.Name.str.replace(r"\(.*\)","")
    df.Rating = pd.to_numeric(df.Rating)
    df.Games = pd.to_numeric(df.Games.str.replace(",",""))
    df.Price = (df.Price.replace(r'[KM]+$', '', regex=True).astype(float) * \
                    df.Price.str.extract(r'[\d\.]+([KM]+)', expand=False)
                    .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
    df.Price = df.Price.astype("int64")
    return df

def dataframe_position_version_deltagames(position, version):
    df0 = dataframe_time_position_version(time_list[0], position, version)
    df1 = dataframe_time_position_version(time_list[1], position, version)
    
    dn = np.zeros(len(df1.index))
    J = np.zeros(len(df1.index))
    i = -1
    for index_df1, row_df1 in df1.iterrows():
        i += 1
        j = -1
        for index_df0, row_df0 in df0.iterrows():
            j += 1
            if J[j] == 1:
                continue
            if row_df1["Name"] == row_df0["Name"] and row_df1["Rating"] == row_df0["Rating"]:
#                 print("%s %s %s %s  - %s %s %s %s " % (row_df1["Name"], row_df1["Rating"], row_df1["Price"], row_df1["Games"],
#                                                        row_df0["Name"], row_df0["Rating"], row_df0["Price"], row_df0["Games"]))
                dn[i] = row_df1["Games"] - row_df0["Games"]
                J[j] = 1
                break
    df1.Games = dn.astype(int)
    return df1

In [14]:
A = 0.6
a = 3e-5 # [price]
b = 7.5e4 # [-]
B = 1
c = 9.5e-6 # [games]
d = 0   # [-]

version_list = ["all_nif", "all_specials"]
position_list = ["GK", "RB,RWB", "LB,LWB", "CB", "CDM,CM,CAM", "RM,RW,RF", "LM,LW,LF", "CF,ST"]
time_list = ["2019121420", "2019121820"]

In [85]:
df = dataframe_position(position_list[3])
df.head(60)

Clément Lenglet  85 257369 40000 - Clément Lenglet  85 20982 41750 
Éder Militão  86 244799 535000 - Éder Militão  81 213583 2100 
Éder Militão  86 244799 535000 - Éder Militão  81 19202 3000 
Éder Militão  81 0 0 - Éder Militão  81 0 0 
Lucas Hernández  84 146904 7300 - Lucas Hernández  84 138884 42750 
Rio Ferdinand  85 108454 799000 - Rio Ferdinand  88 22233 688000 
Matthijs de Ligt  85 73870 14250 - Matthijs de Ligt  86 73553 33500 
Matthijs de Ligt  85 73870 14250 - Matthijs de Ligt  86 39253 24000 
Matthijs de Ligt  86 0 0 - Matthijs de Ligt  86 0 0 
Fernando Hierro Ruiz  86 23088 333000 - Fernando Hierro Ruiz  89 20376 378000 
Ronald Koeman  88 21355 549000 - Ronald Koeman  85 19292 248000 


,Name,Rating,Games_sum,Price_weight,Price_min,Value
36,Éder Militão,86,477584.0,275288.944563,2100,1.000000
3,Raphaël Varane,85,279514.0,148000.000000,148000,0.562255
6,Virgil van Dijk,90,259412.0,485000.000000,485000,0.543708
5,Kalidou Koulibaly,89,313828.0,37000.000000,37000,0.458412
1,Clément Lenglet,85,278351.0,40131.914381,40000,0.410713
2,Lucas Hernández,84,285788.0,24527.587582,7300,0.402599
7,Sergio Ramos,89,229772.0,80000.000000,80000,0.392479
9,Aymeric Laporte,87,232991.0,23750.000000,23750,0.327554
31,Nicolás Otamendi,85,136706.0,240000.000000,240000,0.285720
38,Rio Ferdinand,85,130687.0,780116.231913,688000,0.273910


In [ ]:
# Bench: Ben Yedder, de Jong, Martial
# Varane [Militao] 
# Kante [Sissoko] 1000
# Salah [Dembele]
# Aubameyang [Lucas] 1335
# Mane [Sane] 1517